In [1]:
import pandas as pd
import cv2
from os import walk
import numpy as np
from datetime import timedelta
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

In [2]:
def geo_to_pixel(chart, geo_position, extent):
    lon_0, lon_1, lat_0, lat_1 = extent
    
    x_geo = geo_position[0]
    y_geo = geo_position[1]
    x_geo_range = lon_1 - lon_0
    y_geo_range = lat_1 - lat_0
    
    x_pixel = int((x_geo - lon_0) / x_geo_range * chart.shape[1])
    y_pixel = int((lat_1 - y_geo) / y_geo_range * chart.shape[0])
    
    pos_in_pixel = (x_pixel, y_pixel)
    return pos_in_pixel


def draw_path(chart, path, color=None):
    if not color:
        color = (0, 255, 0)
    image = chart.copy()
    for i in range(len(path) - 1):
        start_point = path[i]
        end_point = path[i+1]
        image = cv2.line(image, start_point, end_point, color, 2)
    return image

def draw_ship(chart, pos, course_in_degree, scale, color):
    x, y = pos
    phi = (course_in_degree - 90) * np.pi / 180
    ship_contour = scale * np.matrix([[30, 0], 
                              [10, -10], 
                              [-30, -10], 
                              [-30, 10], 
                              [10, 10]]).T
    rot = np.matrix([[np.cos(phi), -np.sin(phi)], 
                     [np.sin(phi), np.cos(phi)]])
    ship_contour = np.round(np.matmul(rot, ship_contour)).astype(int)
    
    for i in range(5):
        ship_contour[0, i] += x
        ship_contour[1, i] += y
        
    cv2.fillPoly(chart, pts=[ship_contour.T], color=color)
    
    return chart

def draw_square(chart, pos, course_in_degree, scale, color):
    x, y = pos
    phi = (course_in_degree - 90) * np.pi / 180
    ship_contour = scale * np.matrix([[20, 20], 
                                      [20, -20], 
                                      [-20, -20], 
                                      [-20, 20]]).T
    rot = np.matrix([[np.cos(phi), -np.sin(phi)], 
                     [np.sin(phi), np.cos(phi)]])
    ship_contour = np.round(np.matmul(rot, ship_contour)).astype(int)
    
    for i in range(4):
        ship_contour[0, i] += x
        ship_contour[1, i] += y
        
    cv2.fillPoly(chart, pts=[ship_contour.T], color=color)
    
    return chart

In [3]:
directory = "../nautical_chart/"
df_description = pd.read_csv(directory+"chart_detail.csv")
print(df_description)

resolution, x_km_per_deg, y_km_per_deg, lon_0, lon_1, lat_0, lat_1 = df_description.loc[0, :]
extent = (lon_0, lon_1, lat_0, lat_1)

   res  x_km_per_deg  y_km_per_deg   lon_0   lon_1  lat_0  lat_1
0  100         87.81        111.19 -122.67 -122.22  37.54  38.17


In [4]:
np.random.seed(0)

In [5]:
# collect the latest ship positions (SOG < 1) in the past 10 minutes

chart_path = "../nautical_chart/res_100_lon_-122.67_-122.22_lat_37.54_38.17.jpg"
chart = cv2.imread(chart_path)

filename = "AIS_2020_07_04.csv"
csv_directory = "../processed_data/"

df = pd.read_csv(csv_directory + filename)

ship_dict = {}
for i in range(df.shape[0]):
    MMSI, BaseDateTime, LAT, LON, SOG, COG, Heading = df.loc[i, :]
    if MMSI not in ship_dict:
        ship_dict[MMSI] = []
    ship_dict[MMSI].append([BaseDateTime, LAT, LON, SOG, COG])

for key in ship_dict:
    ship_dict[key].sort(key=lambda x : x[0])

t_start = pd.to_datetime('2020-07-' + filename[-6:-4] + ' 20:50:00')  # beginning of the day
t_end = t_start + timedelta(minutes=10)
ships_10min = {}

for mmsi in ship_dict:
    for (BaseDateTime, LAT, LON, SOG, COG) in ship_dict[mmsi]:
        if SOG < 0.5 and t_start <= pd.to_datetime(BaseDateTime) <= t_end:
            if mmsi not in ships_10min:
                ships_10min[mmsi] = []
            pos = geo_to_pixel(chart, [LON, LAT], extent)
            ships_10min[mmsi] = [pos[0], pos[1], COG]

to_be_clustered = []
for mmsi in ships_10min:
    to_be_clustered.append(ships_10min[mmsi][:2])

In [6]:
# perform KMeans clustering and display all the clusters by color

display_each = False
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(to_be_clustered)

ship_mmsi_list = list(ships_10min.keys())
cm = plt.get_cmap('gist_rainbow')

if display_each:
    for target_label in range(n_clusters):
        image = chart.copy()
        for i in range(len(ship_mmsi_list)):
            mmsi = ship_mmsi_list[i]
            pos = ships_10min[mmsi][:2]
            COG = ships_10min[mmsi][2]
            label = kmeans.labels_[i]
            if label != target_label:
                continue
            color = [int(x*255) for x in cm(label/n_clusters)[:3]]

            image = draw_ship(image, pos, COG, scale=3, color=color)

        image = cv2.resize(image, [image.shape[1] // 10, image.shape[0] // 10])
        cv2.imshow("image", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

else:
    image = chart.copy()
    for i in range(len(ship_mmsi_list)):
        mmsi = ship_mmsi_list[i]
        pos = ships_10min[mmsi][:2]
        COG = ships_10min[mmsi][2]
        label = kmeans.labels_[i]
        color = [int(x*255) for x in cm(label/n_clusters)[:3]]

        image = draw_ship(image, pos, COG, scale=3, color=color)

    image = cv2.resize(image, [image.shape[1] // 10, image.shape[0] // 10])
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
cv2.destroyAllWindows()

In [8]:
# perform hierarchical clustering and display all the clusters by color

display_each = False
n_clusters = 20
# clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage="complete").fit(to_be_clustered)
clustering = AgglomerativeClustering(n_clusters=None, linkage="complete", distance_threshold=400, compute_full_tree=True).fit(to_be_clustered)


ship_mmsi_list = list(ships_10min.keys())
cm = plt.get_cmap('gist_rainbow')

if display_each:
    for target_label in range(clustering.n_clusters_):
        image = chart.copy()
        for i in range(len(ship_mmsi_list)):
            mmsi = ship_mmsi_list[i]
            pos = ships_10min[mmsi][:2]
            COG = ships_10min[mmsi][2]
            label = clustering.labels_[i]
            if label != target_label:
                continue
            color = [int(x*255) for x in cm(label/n_clusters)[:3]]

            image = draw_ship(image, pos, COG, scale=3, color=color)

        image = cv2.resize(image, [image.shape[1] // 10, image.shape[0] // 10])
        cv2.imshow("image", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

else:
    image = chart.copy()
    for i in range(len(ship_mmsi_list)):
        mmsi = ship_mmsi_list[i]
        pos = ships_10min[mmsi][:2]
        COG = ships_10min[mmsi][2]
        label = clustering.labels_[i]
        color = [int(x*255) for x in cm(label/n_clusters)[:3]]

        image = draw_ship(image, pos, COG, scale=3, color=color)

    cv2.imwrite("clusters.png", image)
    image = cv2.resize(image, [image.shape[1] // 10, image.shape[0] // 10])
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
# get the median position of each cluster
clusters = {}
for i in range(len(ship_mmsi_list)):
    mmsi = ship_mmsi_list[i]
    label = clustering.labels_[i]
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(ships_10min[mmsi][0:2])

cluster_medians = []
for label in range(clustering.n_clusters_):
    mean = [0, 0]
    for pos in clusters[label]:
        mean[0] += pos[0]
        mean[1] += pos[1]
    mean[0] /= len(clusters[label])
    mean[1] /= len(clusters[label])
    
    median = []
    min_dist_square = 1000000
    for pos in clusters[label]: 
        if (pos[0] - mean[0])**2 + (pos[1] - mean[1])**2 < min_dist_square:
            min_dist_square = (pos[0] - mean[0])**2 + (pos[1] - mean[1])**2
            median = pos.copy()
    cluster_medians.append(median)

image = chart.copy()
for i in range(len(ship_mmsi_list)):
    mmsi = ship_mmsi_list[i]
    pos = ships_10min[mmsi][:2]
    COG = ships_10min[mmsi][2]
    label = clustering.labels_[i]
    color = [int(x*255) for x in cm(label/n_clusters)[:3]]
    if pos in cluster_medians:
#         color = [0, 0, 255]

        image = draw_ship(image, pos, COG, scale=3, color=color)

cv2.imwrite("medians.png", image)

image = cv2.resize(image, [image.shape[1] // 10, image.shape[0] // 10])
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(cluster_medians)

[[1674, 3409], [2788, 4320], [3118, 3387], [2713, 4017], [1500, 2211], [3353, 4219], [2706, 2857], [1841, 3027], [3562, 4288], [2548, 4687], [2262, 4008], [3057, 3984], [2548, 5527], [2113, 3309], [2254, 2725], [3109, 6478], [2495, 4324], [3602, 1254], [2961, 4913], [3521, 763], [2015, 4034]]


In [10]:
targets_in_ocean = [[50, 3200], [50, 5000], [50, 6800], [1200, 6800], [3800, 6500], [3800, 5300], [3950, 1200], [3300, 100]]

targets = cluster_medians
targets += targets_in_ocean

image = chart.copy()
for target in targets:
    image = draw_square(image, target, 0, scale=2, color=[0, 255, 0])

cv2.imwrite("destinations.png", image)

True